In [1]:
import netket as nk
from numpy.lib.function_base import append
from itertools import permutations, combinations
from netket.vqs.mc import get_local_kernel_arguments, get_local_kernel
import time
import matplotlib.pyplot as plt
import json
import numpy as np
from numpy.linalg import eig
from flax.core.frozen_dict import FrozenDict
import flax
from netket.optimizer.qgt import QGTJacobianPyTree


from typing import Callable, Tuple
from functools import partial

import jax
from jax import numpy as jnp

from netket.stats import statistics as mpi_statistics, mean as mpi_mean, Stats
from netket.utils.types import PyTree
from netket.operator.spin import sigmax, sigmay,sigmaz

import matplotlib.pyplot as plt
from tqdm import tqdm
################################################################
from scipy.sparse.linalg import eigsh
################################################################
import os
import pandas as pd



In [2]:
# We always shorten netket as nk
import netket as nk

# Define a 1d chain
L = 6
g = nk.graph.Hypercube(length=L, n_dim=1, pbc=True)
hi = nk.hilbert.Spin(s=1/2, N=g.n_nodes)
# ha = nk.operator.Heisenberg(hilbert=hi, graph=g)
dis=5
b=dis
a=-dis
Gamma = (b-a) * np.random.random_sample(L) + a

ha = sum([ sigmax(hi,i)*sigmax(hi,(i+1)%L)
         + sigmay(hi,i)*sigmay(hi,(i+1)%L)
         + sigmaz(hi,i)*sigmaz(hi,(i+1)%L)
           for i in range(L)])
################################################################################
################################################################################
################################################################################
ha = ha + sum([Gamma[i]*sigmaz(hi,i) for i in range(L)])

In [3]:
Et = np.random.uniform(low=0, high=L, size=None)
# Et = np.random.randint(0,L, size=1)
# Et = 9
Et


3.7279388944686067

In [ ]:
E, vec = np.linalg.eigh(ha.to_dense())
plt.scatter(np.arange(len(E)),E,s=20)
plt.ylabel('E',fontsize=18)
plt.plot([0,len(E)],[Et,Et],'r')

In [37]:
E

array([-14.76548848, -14.6228794 , -12.70567986, -12.1094269 ,
       -11.59315374, -10.69346998,  -9.97115142,  -9.52932544,
        -8.46819275,  -8.03909155,  -7.57263481,  -7.00011147,
        -6.87246157,  -6.1801361 ,  -5.6421618 ,  -5.55513096,
        -4.77341447,  -4.48427292,  -4.3081206 ,  -4.26943293,
        -4.23946804,  -3.71975021,  -2.41093668,  -2.34739335,
        -2.33781236,  -1.92842623,  -1.58602524,  -1.35121289,
        -1.3438576 ,  -0.80610992,  -0.72343429,  -0.66606371,
         0.35489378,   0.53223975,   0.66612697,   0.85381843,
         1.24007134,   1.50248764,   1.86136102,   2.75992866,
         2.83261212,   3.14618157,   3.17424   ,   3.76077472,
         3.82057189,   4.09068707,   4.3129701 ,   4.91994965,
         4.99762261,   5.58602524,   5.72260524,   5.80580558,
         6.89921789,   7.64015687,   7.74411453,   8.26943293,
         9.10684084,  10.3361816 ,  10.61473346,  11.00011147,
        12.26335913,  14.33781236,  16.1094269 ,  16.35

In [38]:
# and like before, we can jit-compile it!
# compute_energy_jit = jax.jit(compute_energy, static_argnames="model")

# and we precompute the sparse-hamiltonian to avoid the overhead of re-computing them all the time
# hamiltonian_jax = ha.to_pauli_strings().to_jax_operator()
# hamiltonian_sparse = hamiltonian.to_sparse()
# hamiltonian_jax_sparse = hamiltonian_jax.to_sparse()

hamil_Et_jax = (ha-Et).to_pauli_strings().to_jax_operator()
hamil_Et_jax_sparse = hamil_Et_jax.to_sparse()

hamil_Et_sq_jax = ((ha-Et)@(ha-Et)).to_pauli_strings().to_jax_operator()
hamil_Et_sq_jax_sparse = hamil_Et_sq_jax.to_sparse()

The model itself is only a set of instructions on how to initialise the parameters and how to compute the result. 

To initialise the parameters we do the following:

In [39]:
def to_array(model, parameters):
    # begin by generating all configurations in the hilbert space.
    # all_States returns a batch of configurations that is (hi.n_states, N) large.
    all_configurations = hi.all_states()

    # now evaluate the model, and convert to a normalised wavefunction.
    logpsi = model.apply(parameters, all_configurations)
    psi = jnp.exp(logpsi)
    psi = psi / jnp.linalg.norm(psi)
    return psi

### 3.3 Energy
Now, using that function, let's build another one that computes the energy of the mean field state for the given parameters! 
To do that, you can for example convert the hamiltonian to a sparse format and compute the energy

In [40]:
def compute_energy(model, parameters, hamil):
    psi_gs = to_array(model, parameters)
    return psi_gs.conj().T@(hamil@psi_gs)

def loss_fun(model, pre_psi, parameters, ha_Et, ha_Et_sq):
    psi_gs = to_array(model, parameters)
    return psi_gs.conj().T@(ha_Et_sq@psi_gs) - 2*pre_psi.conj().T@(ha_Et@psi_gs)

def compute_variance(model, parameters, hamiltonian_sparse):
    psi_gs = to_array(model, parameters)
    variance = psi_gs.conj().T@(hamiltonian_sparse@hamiltonian_sparse@psi_gs) - (psi_gs.conj().T@(hamiltonian_sparse@psi_gs))**2 
    return variance

In [41]:
from functools import partial

# we use partial to directly jit this function. Jitting the top-most will jit everything inside it as well.
@partial(jax.jit, static_argnames='model')
def compute_and_gradient(model, pre_psi, parameters, ha_Et, ha_Et_sq):
    # grad_fun = jax.value_and_grad(compute_energy, argnums=1)
    grad_fun = jax.jacfwd(loss_fun, argnums=2,holomorphic=True)
    return grad_fun(model, pre_psi, parameters, ha_Et, ha_Et_sq)

In [44]:
model = nk.models.RBM(alpha=1, param_dtype=complex)

parameters = model.init(jax.random.PRNGKey(0), np.ones((hi.size, )))

# logging: you can (if you want) use netket loggers to avoid writing a lot of boilerplate...
# they accumulate data you throw at them
logger = nk.logging.RuntimeLog()
energy = []
from tqdm import tqdm
for i in tqdm(range(40000)):
    # compute energy and gradient
    # energy, gradient = compute_energy_and_gradient(model, parameters, hamiltonian_jax_sparse)
    pre_psi = to_array(model, parameters)
    energy.append(compute_energy(model, parameters, ha))
    gradient = compute_and_gradient(model, pre_psi, parameters, hamil_Et_jax_sparse, hamil_Et_sq_jax_sparse)
    
    # update parameters
    parameters = jax.tree.map(lambda x,y:x-0.01*y, parameters, gradient)
    
    # log energy: the logger takes a step argument and a dictionary of variables to be logged
    # logger(step=i, item={'Energy':energy})

    
compute_variance(model, parameters, ha)    

100%|█████████████████████████████████████| 40000/40000 [07:09<00:00, 93.10it/s]


Array(8.00011374-2.25337683e-16j, dtype=complex128)

In [45]:
energy[-1]

Array(5.30192581+8.45875695e-18j, dtype=complex128)